Ticker selections
=================

In [1]:
# If you would like to refresh your data, please execute the bellow codes.

import pandas as pd
from datetime import datetime
from tqdm import tqdm

from mypo import Loader

DOWNLOAD = False

if DOWNLOAD:
    tickers = pd.read_csv("/app/docs/tutorial/tickers.csv")
    loader = Loader()
    for row in tqdm(tickers.to_dict('record')):
        loader.get(row['Ticker'], expense_ratio=row['ExpenseRatio'] * 0.01)
    loader.save('/app/docs/tutorial/all.bin')

In [2]:
loader = Loader.load('/app/docs/tutorial/all.bin')
loader = loader.since(datetime(2005, 1, 1))
market = loader.get_market()

In [3]:
pd.set_option('display.max_rows', 200)

summary = loader.summary()
summary

,established,names,total_assets,volume,expense_ratio
AGG,2003-09-29,iShares Core U.S. Aggregate Bond ETF,85938724864,6466483,0.0005
FVD,2003-08-27,First Trust Value Line Dividend Index Fund,10207452160,1312757,0.0070
FEZ,2002-10-21,SPDR EURO STOXX 50 ETF,1917581312,1547814,0.0029
SPYV,2000-10-02,SPDR Portfolio S&P 500 Value ETF,8475042304,4683800,0.0004
SPYG,2000-10-02,SPDR Portfolio S&P 500 Growth ETF,9469054976,1754557,0.0004
SPTM,2000-10-10,SPDR Portfolio S&P 1500 Composite Stock Market...,4416926720,385500,0.0003
SHY,2002-07-30,iShares 1-3 Year Treasury Bond ETF,19571066880,2950542,0.0015
IEF,2002-07-30,iShares 7-10 Year Treasury Bond ETF,13733128192,8693385,0.0015
DVY,2003-11-07,iShares Select Dividend ETF,15882481664,1065285,0.0039
TLT,2002-07-30,iShares 20+ Year Treasury Bond ETF,14208894976,15174800,0.0015


In [7]:
from mypo import clustering_tickers, make_combinations
cluster = clustering_tickers(market, n=4)
combinations = make_combinations(cluster)
cluster = cluster.sort_values('class')
cluster

,class
DIA,0
VBK,0
VAW,0
VUG,0
VTV,0
IJR,0
XLK,0
XLB,0
XLI,0
IOO,0


In [10]:
import multiprocessing
from mypo.optimizer import MinimumVarianceOptimizer
import numpy as np

def proc(c):
    optimizer = MinimumVarianceOptimizer(span=50000)
    target_market = market.filter(list(c))
    optimizer.optimize(target_market, at=datetime.today())
    w = optimizer.get_weights()
    r = target_market.get_rate_of_change().to_numpy()
    q = np.dot(np.dot(w, np.cov(r.T)), w.T)
    r = np.dot(w, r.mean(axis=0))
    return target_market.get_tickers(), r, q, w

cores = multiprocessing.cpu_count()
with multiprocessing.Pool(cores) as pool:
    imap = pool.imap(proc, combinations)
    result = list(tqdm(imap, total=len(combinations)))


100%|██████████| 12960/12960 [03:15<00:00, 66.33it/s]


In [25]:
df = pd.DataFrame({
    "c": [r[0] for r in result],
    "r": [r[1] for r in result],
    "q": [r[2] for r in result],
    "w": [r[3] for r in result],
})
df['sharp ratio'] = df["r"] / df["q"]


print(df.sort_values("sharp ratio", ascending=False).head(10))
print(df.sort_values("r", ascending=False).head(10))
print(df.sort_values("q").head(10))


                           c         r             q  sharp ratio
903    [SPYG, SHY, LQD, FXI]  0.000092  5.689978e-07   160.859762
8644    [QQQ, SHY, AGG, VDC]  0.000094  5.829743e-07   160.713584
8668    [QQQ, SHY, LQD, VDC]  0.000094  5.829743e-07   160.706434
2343     [VV, SHY, LQD, FXI]  0.000091  5.649561e-07   160.687803
10383   [IVV, SHY, AGG, FXI]  0.000091  5.638219e-07   160.668289
2319     [VV, SHY, AGG, FXI]  0.000091  5.649223e-07   160.659059
10407   [IVV, SHY, LQD, FXI]  0.000091  5.638540e-07   160.635607
879    [SPYG, SHY, AGG, FXI]  0.000091  5.689091e-07   160.584469
8692    [QQQ, SHY, GLD, VDC]  0.000094  5.829829e-07   160.516194
2367     [VV, SHY, GLD, FXI]  0.000091  5.649225e-07   160.498620
                         c         r         q  sharp ratio
8833  [QQQ, TLT, GLD, BBH]  0.000409  0.000031    13.389883
3937  [VGT, TLT, GLD, BBH]  0.000408  0.000030    13.445604
6241  [XLK, TLT, GLD, BBH]  0.000403  0.000030    13.311848
8849  [QQQ, TLT, GLD, EWZ]  0.0004

In [26]:
import yfinance as yf

tickers = ['XLY', 'XLV', 'VHT', 'FVD', 'QQQ', 'VGT', 'XLK', 'XLF', 'VFH']
tickers += ['RWR', 'EWY', 'FXI', 'EWZ', 'XLU']
tickers += ['SHY', 'TLT', 'BBH', 'GLD', 'TIP', 'LQD']
for t in tickers:
    ticker = yf.Ticker(t)
    print(t, ticker.info['longName'])
    

XLY Consumer Discretionary Select Sector SPDR Fund
XLV Health Care Select Sector SPDR Fund
VHT Vanguard Health Care Index Fund ETF Shares
FVD First Trust Value Line Dividend Index Fund
QQQ Invesco QQQ Trust
VGT Vanguard Information Technology Index Fund ETF Shares
XLK Technology Select Sector SPDR Fund
XLF Financial Select Sector SPDR Fund
VFH Vanguard Financials Index Fund ETF Shares
RWR SPDR Dow Jones REIT ETF
EWY iShares MSCI South Korea ETF
FXI iShares China Large-Cap ETF
EWZ iShares MSCI Brazil ETF
XLU Utilities Select Sector SPDR Fund
SHY iShares 1-3 Year Treasury Bond ETF
TLT iShares 20+ Year Treasury Bond ETF
BBH VanEck Vectors Biotech ETF
GLD SPDR Gold Shares
TIP iShares TIPS Bond ETF
LQD iShares iBoxx $ Investment Grade Corporate Bond ETF


In [ ]:
from mypo.optimizer import MinimumVarianceOptimizer
from mypo.rebalancer import MonthlyRebalancer
from mypo import Runner

target = market.filter(['XLY', 'RWR', 'SHY', 'BBH'])
print(target.get_rate_of_change())
optimizer = MinimumVarianceOptimizer()
runner = Runner(
    assets=[0.25, 0.25, 0.25, 0.25],
    rebalancer=MonthlyRebalancer(optimizer=optimizer, do_re_optimize=True),
    cash=0.5
)

runner.run(
    market=target,
    train_span=200,
    verbose=True
)
report = runner.report()
print(optimizer.get_weights())
print(yearly_total_return(report))
print(max_drawdown(report))
print(max_drawdown_span(report))
report.plot()